<a href="https://colab.research.google.com/github/purang2/prompting-nlp/blob/main/Fewshot_GPT%26T5_StyleTransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Manual Prompting for Compositionality with Large Language Models 


**[ Code from ]** [Eunchan Lee of UNIST-LIL-LAB](https://github.com/purang2)


Option
```(python)
- LLM : "Meta AI/opt-6.7b"
- Number of data: 2개

- Data [0]
- Q: '[A man on a bike] [vs] [a bike on a man]. Please Explain what is more natural.', 
- A: '[N]: man, bike [V]: on [SEP] In nature, man can ride bike but bike cannot ride man. [Answer] [A man on a bike] is more natural.'
- Data [1]
- Q: '[The grass is eating the horse] [vs] [The horse is eating the grass.] Please Explain what is more natural.'
- A: '[N]: grass, horse [V]: eat [SEP] In nature, horse can eat the grass because horses are animal but grass cannot eat horse because grass is a plant. [Answer] [The horse is eating the grass] is more natural.'
```

install
- torch
- pip install transformers

In [1]:
# GPU 사용 가능 -> True, GPU 사용 불가 -> False
import os
import torch
print(torch.cuda.is_available())
os.environ["CUDA_VISIBLE_DEVICES"]= "0"

True


In [2]:
import gc
import torch 
  
#gc.collect()
#torch.cuda.empty_cache()

In [3]:
#import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

/home/chanlee/bin/anaconda3/envs/comp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Put datapair

In [11]:
few_src = [] 
few_tgt = []


def put_datapair(src, tgt):
  few_src.append(src)
  few_tgt.append(tgt)


# k=4
'''
put_datapair("k", "x")
put_datapair("kk", "xx")
put_datapair("kkk", "xxx")
put_datapair("kkkk", "xxxx")
'''


# k=4 -> Manual example: Text style transfer
#put_datapair(A,B): Style : A--> B
'''
put_datapair("South Korea's Capital is Seoul", "Seoul -> South Korea")
put_datapair("Mexico's Capital is Mexico City", "Mexico City -> Mexico")
put_datapair("Germany's Capital is Berlin", "Berlin -> Germany")
put_datapair("Uganda's Capital is Kampala", "Kampala -> Uganda")
'''
#put_datapair("What's difference between \" a man on a bike \" vs \"a bike on a man\" ?","In \"a man on a bike,\" the preposition \"on\" suggests that the man is riding or sitting on the bike. In this sentence, the bike is the means of transportation, and the man is the one using it. In contrast, \"a bike on a man\" reverses the relationship between the nouns, implying that the bike is positioned on top of the man.")
#put_datapair("What's difference between \" a brown cat is looking at a gray dog and sitting in a white bathtub\" vs \"a gray cat is looking at a white dog and sitting in a brown bathtub\"?", "\"A brown cat is looking at a gray dog and sitting in a white bathtub\": This sentence describes a scene where a brown-colored cat is looking at a gray-colored dog while sitting in a white-colored bathtub. In contrast \"A gray cat is looking at a white dog and sitting in a brown bathtub\": This sentence describes where a gray-colored cat is looking at a white-colored dog while sitting in a brown-colored bathtub.")
#put_datapair("What's difference between \"a red paved road behind the white house\" vs \"a red house behind the white paved road\"?", "\"A red paved road behind the white house\": This sentence describes a scenario where there is a road with red paving located behind a white house. The main focus of the description is the position of the red paved road in relation to the white house. \"A red house behind the white paved road\": This sentence describes a different scenario where there is a red house located behind a road with white paving. In this case, the main focus of the description is the position of the red house in relation to the white paved road.")

put_datapair("[A man on a bike] [vs] [a bike on a man]. Please Explain what is more natural", "[N]: man, bike [V]: on [SEP] In nature, man can ride bike but bike cannot ride man. [Answer] [A man on a bike] is more natural.")
put_datapair("[The grass is eating the horse] [vs] [The horse is eating the grass.] Please Explain what is more natural", "[N]: grass, horse [V]: eat [SEP] In nature, horse can eat the grass because horses are animal but grass cannot eat horse because grass is a plant. [Answer] [The horse is eating the grass] is more natural.")


print(few_src)
print(few_tgt)


['[A man on a bike] [vs] [a bike on a man]. Please Explain what is more natural.', '[The grass is eating the horse] [vs] [The horse is eating the grass.] Please Explain what is more natural.']
['[N]: man, bike [V]: on [SEP] In nature, man can ride bike but bike cannot ride man. [Answer] [A man on a bike] is more natural.', '[N]: grass, horse [V]: eat [SEP] In nature, horse can eat the grass because horses are animal but grass cannot eat horse because grass is a plant. [Answer] [The horse is eating the grass] is more natural.']


### Prompt

***Reference: [A Recipe for Arbitrary Text Style Transfer with Large Language Models, (Reif et al. (Google Research), ACL 2022)](https://aclanthology.org/2022.acl-short.94/)***

In [12]:
in_context_text = ''

for i in range(len(few_src)):
  #in_context_text += f'Here is some text: {few_src[i]}. Here is a rewrite of the text, which is more simple: {few_tgt[i]}\n'
  in_context_text += f'Q: {few_src[i]}. \nA: {few_tgt[i]}\n\n'
  
print(in_context_text)

Q: [A man on a bike] [vs] [a bike on a man]. Please Explain what is more natural.. 
A: [N]: man, bike [V]: on [SEP] In nature, man can ride bike but bike cannot ride man. [Answer] [A man on a bike] is more natural.

Q: [The grass is eating the horse] [vs] [The horse is eating the grass.] Please Explain what is more natural.. 
A: [N]: grass, horse [V]: eat [SEP] In nature, horse can eat the grass because horses are animal but grass cannot eat horse because grass is a plant. [Answer] [The horse is eating the grass] is more natural.




transformers model

In [6]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM
from transformers import pipeline


# GPT-2 (This is the smallest version of GPT-2, with 124M parameters.) = "gpt2"
# GPT-2 (XL) (1.5B parameters.) = "gpt2-xl"
# T5 (base, with 220 million parameters.) = "t5-base"
# T5 (3B parameters.) = "t5-3b"
# T5 (11B parameters.) = "t5-11b" -> 45.2GB (Colab may not cover this size..)
# GPT-J (6B parameters.) = "EleutherAI/gpt-j-6B"

# In-context learning may work well at least 1 billions parameters model. -eunchan-

#model_zoo = ["gpt2","gpt2-xl","t5-base","t5-3b","t5-11b","EleutherAI/gpt-j-6B"]
model_zoo = ["bigscience/bloom-7b1","bigscience/test-bloomd-6b3","bigscience/T0pp","lmsys/vicuna-13b-delta-v1.1","decapoda-research/llama-7b-hf","facebook/opt-6.7b"]

model_name = model_zoo[5]


if model_name.find('t5') > -1 or model_name.lower().find('t0') > -1: #model = t5, t0 [encoder-decoder]
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
  nlg_pipeline = pipeline('text2text-generation',model=model, tokenizer=tokenizer)

else: #model = gpt,etc [decoder-only]
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForCausalLM.from_pretrained(model_name)
  #nlg_pipeline = pipeline(model=model, tokenizer=tokenizer)

Loading checkpoint shards: 100%|██████████| 2/2 [00:14<00:00,  7.12s/it]


model inference

In [13]:

#test_input_text = 'kjkjkjkj'
#test_input_text = "What's difference between \"the horse is eating the grass\" vs \"the grass is eating the horse\"?"
test_input_text = "[The coffee is drinking a boy] [vs] [A boy is drinkng the coffee.] Please Explain what is more natural"
#test_input_text = "Italy's Capital is Roma"
test_output_length = 100 #token length


if model_name.find('t5') > -1 or model_name.lower().find('t0') > -1: #model = t5, t0 [encoder-decoder]
  def generate_text(pipe, text, num_return_sequences=5, max_length=512):
    text = f"{text}"
    out = pipe(text, num_return_sequences=num_return_sequences, max_length=max_length, num_beams=5, no_repeat_ngram_size=2,)
    return [x['generated_text'] for x in out]

  #target_text = 'kjkjkjkj'
  #src_text = in_context_text + f"Here is some text: {test_input_text}. Here is a rewrite of the text, which is more simple: "
  src_text = in_context_text + f"Q: {test_input_text} \nA: "
  #src_text = f"Q: {test_input_text}. \nA: "
  

  print("Input text:", src_text)
  test_output_text = generate_text(nlg_pipeline, src_text, num_return_sequences=1, max_length=test_output_length)


  #you can cook this output anything you want!  
  print(test_output_text)



else:  #model = GPT, ETC
  src_text = in_context_text + f"Q: {test_input_text} \nA:"
  tokens = tokenizer.encode(src_text, return_tensors='pt')
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=len(tokens[0])+test_output_length)
  generated = tokenizer.batch_decode(gen_tokens)[0]
  
  test_output_text = generated[len(src_text):]
  #print(generated)

  #you can cook this output anything you want!
  print("OUTPUT\n\n",test_output_text)


OUTPUT

  
A: [N]: boy, coffee [V]: drink [SEP] In nature, boy can drink coffee because boy is a human being but coffee cannot drink boy because coffee is a drink. [Answer] [The boy is drinking the coffee] is more natural.</s>
